# Class Parser Workflow Test

This notebook reproduces the workflow from poc.ipynb step by step using the new src/ structure.
Test each component individually and then run the complete pipeline.

## Workflow Overview
1. **Data Extraction** - Extract slides and plan from PDF (deterministic)
2. **Branch A** - With plan: outline_two_pass → mapping_two_pass → enrich → writer
3. **Branch B** - No plan: outline_one_shot → enrich → writer
4. **Export** - Generate DOCX document


In [39]:
# Dev hot-reload setup (no kernel restart needed)
%load_ext autoreload
%autoreload 2

import sys, importlib
from pathlib import Path
# Ensure project root is on sys.path
sys.path.insert(0, str(Path('.').resolve()))

import src
import src.models as models
import src.llm as llm
import src.data_extraction as data_extraction
import src.course as course_mod
import src.pipeline as pipeline_mod


def refresh_imports():
    """Reload modules and rebind notebook aliases."""
    importlib.reload(src)
    importlib.reload(models)
    importlib.reload(llm)
    importlib.reload(data_extraction)
    importlib.reload(course_mod)
    importlib.reload(pipeline_mod)
    
    globals().update({
        # Models
        "Content": models.Content,
        "ContentSection": models.ContentSection,
        "MappingItem": models.MappingItem,
        "SectionSlideMapping": models.SectionSlideMapping,
        "Slides": models.Slides,
        "OutlineAndMapping": models.OutlineAndMapping,
        "CourseMetadata": models.CourseMetadata,
        # LLM
        "OutlineOneShot": llm.OutlineOneShot,
        "OutlineTwoPass": llm.OutlineTwoPass,
        "MappingTwoPass": llm.MappingTwoPass,
        "Writer": llm.Writer,
        "PromptManager": llm.PromptManager,
        # Deterministic extractors
        "SlidesExtractor": data_extraction.SlidesExtractor,
        "PlanExtractor": data_extraction.PlanExtractor,
        # Orchestrators
        "Course": course_mod.Course,
        "CoursePipeline": pipeline_mod.CoursePipeline,
    })
    print("🔁 Imports refreshed")

# Initial bind
refresh_imports()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 Imports refreshed


## Step 1: Data Extraction - Slides

Extract slides from PDF using the exact logic from poc.ipynb


In [40]:
slides_path = "/Users/youssefjanjar/Documents/formascience/class_parser/volume/slides/"
import os 
chapters = os.listdir(slides_path)
for index, chapter in enumerate(chapters):
    print(f"id {index} : {chapter}")

id 0 : cours_4_plan.pdf
id 1 : .DS_Store
id 2 : Ch6-Code génétique et traduction.pdf
id 3 : cours_1.pdf
id 4 : cours_4.pdf
id 5 : Ch1-Sucres et Lipides.pdf
id 6 : Ch3_Métabolisme des glucides.pdf
id 7 : Ch2-Acides aminés et protéines.pdf
id 8 : Ch7.1-Génome humain.pdf
id 9 : Ch7.2-Techniques de biologie moléculaire.pdf
id 10 : Ch4-Plan du polycopié.docx
id 11 : Ch4-Structure, réplication et réparation de l_ADN.pdf
id 12 : Ch5-Transcription, maturation et régulation.pdf


In [41]:
# Test slides extraction
pdf_path = slides_path+chapters[12]

# Initialize extractor
slides_extractor = SlidesExtractor(
    min_avg_len=10,
    max_lines=20,
    merge_tol=2.0
)

print(f"🔍 Extracting slides from: {pdf_path}")

try:
    # Extract slides as Slides objects
    slides = slides_extractor.extract_slides(pdf_path)
    print(f"✅ Extracted {len(slides)} slides")
    
    # Display first few slides
    print("\n📋 First 3 slides:")
    for i, slide in enumerate(slides[:3]):
        print(f"\n--- Slide {i+1}: {slide.id} ---")
        print(f"Title: {slide.title}")
        print(f"Content (first 200 chars): {slide.content[:200]}...")
        print("=" * 50)
        
except FileNotFoundError:
    print(f"❌ PDF file not found: {pdf_path}")
    print("Please ensure the PDF file exists in ./volume/slides/")
    slides = []  # Empty list for testing
except Exception as e:
    print(f"❌ Error extracting slides: {e}")
    slides = []


🔍 Extracting slides from: /Users/youssefjanjar/Documents/formascience/class_parser/volume/slides/Ch5-Transcription, maturation et régulation.pdf
✅ Extracted 109 slides

📋 First 3 slides:

--- Slide 1: SL_001 ---
Title: Transcription de l’ADN en ARN
Content (first 200 chars): Transcription de l’ADN en ARN
1
1) Mécanismes généraux de la transcription
2) Maturation des ARN chez les eucaryotes
3) Régulation de la transcription chez les eucaryotes
Texte en italique : informati...

--- Slide 2: SL_002 ---
Title: 1. Mécanismes généraux de la transcription
Content (first 200 chars): 1. Mécanismes généraux de la transcription
2
1.1. Définitions: Transcription – Gène – Différents types d’ARN
1.2. Machinerie transcriptionnelle : les ARN polymérases
3 ARN polymérases nucléaires chez ...

--- Slide 3: SL_003 ---
Title: 1.1. Définition: Transcription chez les eucaryotes
Content (first 200 chars): 1.1. Définition: Transcription chez les eucaryotes
3
Place de la transcription:
Le ‘dogme central’ de l

In [42]:
import json
import tiktoken

# Dump slides to JSON
slides_json = json.dumps([slide.model_dump() for slide in slides], ensure_ascii=False, indent=2)

# Calculate tokens using GPT-4 encoding (tiktoken doesn't have GPT-5 yet, using GPT-4 as closest)
encoding = tiktoken.encoding_for_model("gpt-4")
token_count = len(encoding.encode(slides_json))

print(f"📊 Slides JSON dump:")
print(f"Number of slides: {len(slides)}")
print(f"JSON size: {len(slides_json)} characters")
print(f"Estimated tokens (GPT-4 encoding): {token_count}")

# Display the JSON (truncated for readability)
print(f"\n📋 JSON content (first 1000 chars):")
print(slides_json[:1000] + "..." if len(slides_json) > 1000 else slides_json)

📊 Slides JSON dump:
Number of slides: 109
JSON size: 76526 characters
Estimated tokens (GPT-4 encoding): 24578

📋 JSON content (first 1000 chars):
[
  {
    "id": "SL_001",
    "title": "Transcription de l’ADN en ARN",
    "content": "Transcription de l’ADN en ARN\n1\n1) Mécanismes généraux de la transcription\n2) Maturation des ARN chez les eucaryotes\n3) Régulation de la transcription chez les eucaryotes\nTexte en italique : informations complémentaires non requises pour l’examen\nFigures avec légendes en italique: montrées à titre d’illustration (ne sont pas à apprendre pour l’examen)\nU.E.2_Organisation et expression du génome humain\nDidier Devys"
  },
  {
    "id": "SL_002",
    "title": "1. Mécanismes généraux de la transcription",
    "content": "1. Mécanismes généraux de la transcription\n2\n1.1. Définitions: Transcription – Gène – Différents types d’ARN\n1.2. Machinerie transcriptionnelle : les ARN polymérases\n3 ARN polymérases nucléaires chez l’homme = 3 classes de gènes\n1

In [43]:
course_metadata = CourseMetadata(
    name="Transcription, maturation et régulation de l'ADN",
    course_title="Transcription, maturation et régulation de l'ADN",
    level="L1", 
    block="SANTE",
    semester="S1",
    subject="UE2 - Les molécules du vivant",
    chapter="5",
)

# Single Shot

# Single shot:  Extract the plan for this class 

In [44]:
outline_one_shot = OutlineOneShot()

outline, mapping = outline_one_shot.build_outline_and_mapping(slides, course_metadata)

print(outline.print_outline())

Course Outline:
Mécanismes généraux de la transcription
  Définitions et principes de la transcription (types d'ARN, différences ADN/ARN, sens de transcription)
  Machinerie transcriptionnelle : ARN polymérases (mitochondriale et nucléaires) et classes de gènes
  Gènes de classe II : organisation et régulation (TSS, exons/introns, îlots CpG, méthylation)
  Initiation par ARN Pol II : promoteur minimum, facteurs généraux et formation du complexe de pré-initiation
  Transition initiation → élongation et élongation de la transcription (CTD, pauses, vitesse, proofreading)
Maturation des ARN chez les eucaryotes
  Modifications et maturation des pré-ARNm
    Coiffage 5’ : formation et fonctions de la coiffe
    Épissage : sites consensus, mécanismes et couplage à la transcription
    Clivage 3’ et polyadénylation : signal AAUAAA, PAP, PABP
  Terminaison de la transcription des pré-ARNm et conséquences pour la maturation
  Diversification du protéome (promoteurs alternatifs, épissage alternat

In [45]:
print(mapping.visualize_mapping(outline))

Section-to-Slides Mapping:

[Root] SEC_1
Title: Mécanismes généraux de la transcription
Slides: 34 slide(s)
Slide IDs: SL_003, SL_004, SL_005, SL_006, SL_007, SL_008, SL_009, SL_010, SL_011, SL_012, SL_013, SL_014, SL_015, SL_016, SL_017, SL_018, SL_019, SL_026, SL_027, SL_020, SL_021, SL_022, SL_023, SL_024, SL_025, SL_031, SL_032, SL_033, SL_034, SL_035, SL_036, SL_037, SL_038, SL_039

  [Level 1] SEC_1.1
  Title: Définitions et principes de la transcription (types d'ARN, différences ADN/ARN, sens de transcription)
  Slides: 5 slide(s)
  Slide IDs: SL_003, SL_004, SL_005, SL_006, SL_007

  [Level 1] SEC_1.2
  Title: Machinerie transcriptionnelle : ARN polymérases (mitochondriale et nucléaires) et classes de gènes
  Slides: 8 slide(s)
  Slide IDs: SL_008, SL_009, SL_010, SL_011, SL_012, SL_013, SL_014, SL_015

  [Level 1] SEC_1.3
  Title: Gènes de classe II : organisation et régulation (TSS, exons/introns, îlots CpG, méthylation)
  Slides: 6 slide(s)
  Slide IDs: SL_016, SL_017, SL_01

# Single shot: Writing content from mapping and the outline

In [46]:
outline_with_slides = outline.enrich_with_slides(slides=slides, mapping=mapping)
writer = Writer()

course = writer.write_course(enriched_content=outline_with_slides)

type(course)

src.models.Content

# Store a course as a artifact that I can load again into Course

In [47]:
type(course)

src.models.Content

In [48]:
print(course.print_content())

Course Content:

[Root] ID: SEC_1
Title: Mécanismes généraux de la transcription
Content:
  [1] Cette partie présente les principes essentiels de la transcription chez
      les eucaryotes : définitions, machinerie enzymatique, organisation des
      gènes et étapes clés (initiation, élongation, terminaison).
  [2] • Définitions et types d'ARN (ARNm, ARNt, ARNr) et notion de pré-ARN et
      maturation
  [3] • ARN polymérases : Pol I, Pol II, Pol III (nucléaires) et POLRMT
      (mitochondriale)
  [4] • Organisation des gènes de classe II : TSS, promoteur minimum,
      exons/introns, îlots CpG
  [5] • Initiation par Pol II : facteurs généraux, assemblage du PIC, rôle du
      TFIIH et phosphorylation du CTD
  [6] • Transition initiation → élongation : pauses, kinases CDK7/CDK9, vitesse
      et fidélité de l'élongation

  [Level 1] ID: SEC_1.1
  Title: Définitions et principes de la transcription (types d'ARN, différences ADN/ARN, sens de transcription)
  Content:
    [1] Définition :

In [49]:
try:
    pilot_course = Course(
        name=course_metadata.name or course_metadata.course_title,
        course_title=course_metadata.course_title,
        level=course_metadata.level,
        block=course_metadata.block,
        semester=course_metadata.semester,
        subject=course_metadata.subject,
        chapter=course_metadata.chapter,
        content=course,  # Direct assignment should work
    )
    
    print("✅ Course created successfully!")
    
except Exception as e:
    print(f"❌ Error creating course: {e}")
    
    # Alternative approach - create Course without content first, then assign
    try:
        pilot_course = Course(
            name=course_metadata.name or course_metadata.course_title,
            course_title=course_metadata.course_title,
            level=course_metadata.level,
            block=course_metadata.block,
            semester=course_metadata.semester,
            subject=course_metadata.subject,
            chapter=course_metadata.chapter,
            # content=None by default
        )
        
        # Assign content directly after creation
        pilot_course.content = course
        
        print("✅ Course created successfully (alternative method)!")
        
    except Exception as e2:
        print(f"❌ Alternative method also failed: {e2}")
        
        # Last resort - recreate Content from dict
        try:
            course_dict = course.model_dump()
            fresh_content = Content.model_validate(course_dict)
            
            pilot_course = Course(
                name=course_metadata.name or course_metadata.course_title,
                course_title=course_metadata.course_title,
                level=course_metadata.level,
                block=course_metadata.block,
                semester=course_metadata.semester,
                subject=course_metadata.subject,
                chapter=course_metadata.chapter,
                content=fresh_content,
            )
            
            print("✅ Course created successfully (fresh content method)!")
            
        except Exception as e3:
            print(f"❌ All methods failed: {e3}")

❌ Error creating course: 1 validation error for Course
content
  Input should be a valid dictionary or instance of Content [type=model_type, input_value=Content(sections=[Content... 'SL_110', 'SL_111'])])), input_type=Content]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
✅ Course created successfully (alternative method)!


In [50]:
print(pilot_course.print_outline())

Course: Transcription, maturation et régulation de l'ADN
Level: L1
Block: SANTE
Semester: S1
Subject: UE2 - Les molécules du vivant
Chapter: 5


Course Outline:
Mécanismes généraux de la transcription
  Définitions et principes de la transcription (types d'ARN, différences ADN/ARN, sens de transcription)
  Machinerie transcriptionnelle : ARN polymérases (mitochondriale et nucléaires) et classes de gènes
  Gènes de classe II : organisation et régulation (TSS, exons/introns, îlots CpG, méthylation)
  Initiation par ARN Pol II : promoteur minimum, facteurs généraux et formation du complexe de pré-initiation
  Transition initiation → élongation et élongation de la transcription (CTD, pauses, vitesse, proofreading)
Maturation des ARN chez les eucaryotes
  Modifications et maturation des pré-ARNm
    Coiffage 5’ : formation et fonctions de la coiffe
    Épissage : sites consensus, mécanismes et couplage à la transcription
    Clivage 3’ et polyadénylation : signal AAUAAA, PAP, PABP
  Termin

In [51]:
pilot_course.save_to_json(output_path="volume/artifacts/json")

💾 Course saved to: volume/artifacts/json/transcription,_maturation_et_régulation_de_l'adn_20250821_093019.json


"volume/artifacts/json/transcription,_maturation_et_régulation_de_l'adn_20250821_093019.json"

In [38]:
pilot_course.to_docx(output_path="volume/artifacts/docx", template_path="volume/templates/fs_template.docx")

📄 Course exported to DOCX: volume/artifacts/docx/Chapitre_6_code_genetique_et_traduction.docx


'volume/artifacts/docx/Chapitre_6_code_genetique_et_traduction.docx'

In [32]:
# Check what's actually in your fresh_content variable
print(f"Type: {type(fresh_content)}")
print(f"Is Content? {isinstance(fresh_content, Content)}")

# Check the sections
if hasattr(fresh_content, 'sections'):
    print(f"Sections type: {type(fresh_content.sections)}")
    if fresh_content.sections:
        print(f"First section type: {type(fresh_content.sections[0])}")
        print(f"First section: {fresh_content.sections[0]}")
else:
    print("No sections attribute")

# Check if it's the right Content class
print(f"Content class: {Content}")
print(f"fresh_content's content class: {type(fresh_content).__module__}.{type(fresh_content).__name__}")

Type: <class 'src.models.Content'>
Is Content? True
Sections type: <class 'list'>
First section type: <class 'src.models.ContentSection'>
First section: id='SEC_1' title='Présentation du module et plan du cours' content=["Ce module porte sur l'organisation du génome humain et sur les méthodes biotechnologiques permettant de le caractériser et de l'analyser. Il vise à donner une vision intégrée de la composition, de l'architecture et de la dynamique des génomes, ainsi que des conséquences fonctionnelles et médicales des variations génomiques.", "Le plan du cours présente successivement une introduction aux concepts évolutifs et aux organismes modèles, les notions fondamentales de génétique moléculaire, la description du génome humain (nucléaire et mitochondrial), le projet de séquençage, l'architecture globale du génome, la description des éléments constitutifs (gènes codants et non codants, pseudogènes, éléments répétés), la comparaison intergénomique, les différents types de variation

# Two-shots

# Two-shots

## Plan from page

Extract plan text from PDF for Branch A (two-pass approach)


## Extract the slides

In [70]:
pdf_path = './volume/slides/cours_1.pdf'
plan_path = './volume/slides/cours_4_plan.pdf'

extractor = SlidesExtractor()

slides = extractor.extract_slides(pdf_path=pdf_path)

slides[:5]



2025-08-18 10:12:25,875 [DEBUG] src.data_extraction.slides_extractor: Starting slide extraction from ./volume/slides/cours_1.pdf
2025-08-18 10:12:25,924 [DEBUG] src.data_extraction.slides_extractor: PDF opened, found 111 pages
2025-08-18 10:12:26,844 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P412' is an invalid float value
2025-08-18 10:12:26,845 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P414' is an invalid float value
2025-08-18 10:12:26,861 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P444' is an invalid float value
2025-08-18 10:12:26,876 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P451' is an invalid float value
2025-08-18 10:12:26,877 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P455' is an invalid float value
2025-08-18 10:12:26,894 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P465' is an invalid float value

[Slides(id='SL_001', title='7.1 Architecture du génome', content='L1SpS: UE 2 Les molécules du vivant\nU.E.2 Les molécules du vivant\n7. Organisation du génome humain, méthodes en\nbiotechnologie\n7.1 Architecture du génome\nhumain\nJean Muller\nLaboratoire Diagnostic Génétique (HUS)\nLaboratoire de Génétique médicale (Inserm U1112)\njeanmuller@unistra.fr'),
 Slides(id='SL_002', title='Plan du cours', content='L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2'),
 Slides(id='SL_003', title='Introduction', content='L1SpS: UE 2

In [74]:
# Test plan extraction
plan_extractor = PlanExtractor()

plan_txt = plan_extractor.extract_plan_from_page(pdf_path=pdf_path, page_number=2)

print(plan_txt)

2025-08-18 10:13:00,872 [DEBUG] src.data_extraction.plan_extractor: Starting plan extraction from ./volume/slides/cours_1.pdf (page 2)
2025-08-18 10:13:00,928 [DEBUG] src.data_extraction.plan_extractor: PDF opened, found 111 pages
2025-08-18 10:13:00,948 [DEBUG] src.data_extraction.plan_extractor: Successfully extracted plan from page 2 (1599 chars)
   L1SpS: UE 2 Les molécules du vivant                                                             
      Plan      du     cours                                                                       
        •  Introduction                                                                            
             •  Notions  fondamentales                                                             
        •  Le génome    humain                                                                     
             •  Projet de séquençage                                                               
             •  Architecture globale            

## Build the Outline

In [75]:
out_builder = OutlineTwoPass()

outline = out_builder.build_outline(plan_txt)

2025-08-18 10:13:26,958 [DEBUG] src.llm.outline_two_pass: Starting outline generation from plan text (1599 chars)
2025-08-18 10:13:41,765 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-18 10:13:41,817 [DEBUG] src.llm.outline_two_pass: Successfully generated outline with 6 sections


In [78]:
print(outline.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


## Build the mapping

In [79]:
slides = slides[:10]

In [80]:
mapping_builder = MappingTwoPass()

mapping = mapping_builder.build_mapping(slides=slides, outline=outline) 

type(mapping)

2025-08-18 10:14:02,625 [DEBUG] src.llm.mapping_two_pass: Starting mapping generation for 10 slides and 6 sections
2025-08-18 10:15:20,967 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-18 10:15:20,996 [DEBUG] src.llm.mapping_two_pass: Successfully generated mapping with 13 entries


src.models.SectionSlideMapping

In [82]:
print(mapping.visualize_mapping(outline))

Section-to-Slides Mapping:

[Root] SEC_1
Title: Introduction
Slides: 2 slide(s)
Slide IDs: SL_002, SL_003

  [Level 1] SEC_1.1
  Title: Notions fondamentales
  Slides: 7 slide(s)
  Slide IDs: SL_004, SL_005, SL_006, SL_007, SL_008, SL_009, SL_010

[Root] SEC_2
Title: Le génome humain
Slides: 1 slide(s)
Slide IDs: SL_001

  [Level 1] SEC_2.1
  Title: Projet de séquençage
  Slides: 1 slide(s)
  Slide IDs: SL_002

  [Level 1] SEC_2.2
  Title: Architecture globale
  Slides: 1 slide(s)
  Slide IDs: SL_001

    [Level 2] SEC_2.2.1
    Title: Génome mitochondrial
    Slides: 1 slide(s)
    Slide IDs: SL_001

    [Level 2] SEC_2.2.2
    Title: Génome nucléaire
    Slides: 1 slide(s)
    Slide IDs: SL_001

  [Level 1] SEC_2.3
  Title: Description des principaux éléments constituants
  Slides: 1 slide(s)
  Slide IDs: SL_001

    [Level 2] SEC_2.3.1
    Title: Gènes (codants et non-codants), pseudogènes et éléments répétés
    Slides: 1 slide(s)
    Slide IDs: SL_001

[Root] SEC_3
Title: Comparai

In [87]:
course_with_slides = outline.enrich_with_slides(slides=slides, mapping=mapping)

print(course_with_slides.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


## Write the course

In [84]:
writer = Writer()

course = writer.write_course(enriched_content=course_with_slides)
type(course)

2025-08-18 10:16:07,926 [DEBUG] src.llm.writer: Starting content enhancement for 6 sections
2025-08-18 10:19:58,868 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-18 10:19:58,980 [DEBUG] src.llm.writer: Successfully enhanced content with 6 sections


src.models.Content

In [89]:
type(course)

src.models.Content

In [88]:
test_course = Course(
    name="Architecture du génome humain",
    course_title="Architecture du génome humain",
    level="L1",
    block="SANTE",
    semester="S1",
    subject="UE-1 Constitution et transformation de la matière",
    content=course
)

print(test_course.print_outline())

ValidationError: 1 validation error for Course
content
  Input should be a valid dictionary or instance of Content [type=model_type, input_value=Content(sections=[Content...ides=None, mapping=None), input_type=Content]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [90]:
from src.models import Content as ModelsContent
print("Course content annotation ->", ModelsContent, ModelsContent.__module__)

print("Returned content instance ->", type(course), type(course).__module__)

print("Same class object? ->", type(course) is ModelsContent)

Course content annotation -> <class 'src.models.Content'> src.models
Returned content instance -> <class 'src.models.Content'> src.models
Same class object? -> False


## Tests


In [64]:
refresh_imports()

from src.models import PipelineConfig

config = PipelineConfig.load("pipeline.yaml")
config.inputs, config.outputs, config.metadata

🔁 Imports refreshed


({'slides_pdf': '/Users/youssefjanjar/Documents/formascience/class_parser/volume/slides/cours_1.pdf',
  'plan_page': 2},
 {'save_json': True,
  'save_docx': True,
  'template_path': '/Users/youssefjanjar/Documents/formascience/class_parser/volume/templates/fs_template.docx',
  'output_dir': '/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts'},
 CourseMetadata(name='Architecture du génome humain', course_title='Architecture du génome humain', level='L1', block='SANTE', semester='S1', subject='UE-1 - Les bases de la biologie', chapter=None, year=2024, professor='Youssef Janjar'))

In [65]:
import os 
os.listdir(config.outputs['output_dir'])

['architecture_du_génome_humain_20250817_224951.json',
 '.DS_Store',
 'architecture_du_génome_humain_filled.docx',
 'docx']

In [66]:
from src.course import Course

course = Course.load_from_json(config.outputs['output_dir']+'/architecture_du_génome_humain_20250817_224951.json')

print(course.content.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


In [67]:
course.save_to_json(output_path=config.outputs['output_dir'])

💾 Course saved to: /Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/architecture_du_génome_humain_20250818_082858.json


'/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/architecture_du_génome_humain_20250818_082858.json'

In [61]:
course.to_docx(template_path=config.outputs['template_path'], output_path=config.outputs['output_dir']+'/docx')

📄 Course exported to DOCX: /Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/docx/architecture_du_génome_humain.docx


'/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/docx/architecture_du_génome_humain.docx'

# Testing the pipeline

In [68]:
import os
from pathlib import Path

# Print current working directory
print("Current working directory:", os.getcwd())

# Print root folder using pathlib
root_path = Path.cwd()
print("Root folder (pathlib):", root_path)

# Print absolute path
print("Absolute path:", root_path.absolute())

# List contents of root folder
print("\nContents of root folder:")
for item in root_path.iterdir():
    if item.is_dir():
        print(f"📁 {item.name}/")
    else:
        print(f"📄 {item.name}")


Current working directory: /Users/youssefjanjar/Documents/formascience/class_parser
Root folder (pathlib): /Users/youssefjanjar/Documents/formascience/class_parser
Absolute path: /Users/youssefjanjar/Documents/formascience/class_parser

Contents of root folder:
📁 .cursor/
📄 .DS_Store
📄 create_structure.sh
📁 audio_extractor/
📄 pyproject.toml
📄 data_structure.md
📁 tests/
📁 __pycache__/
📄 README.md
📁 .mypy_cache/
📄 .gitignore
📄 diagram.mermaid
📄 .env
📁 .venv/
📄 pipeline.yaml
📄 poetry.lock
📁 .git/
📄 main.py
📄 test_workflow.ipynb
📁 data/
📁 volume/
📁 notebooks/
📁 src/


In [2]:
import logging
import sys
from pathlib import Path

from src.pipeline import CoursePipeline
from src.models import CourseMetadata

# Configure logging level
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG for more detailed logs, WARNING for less
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # Console output
        logging.FileHandler('pipeline.log')  # Optional: save to file
    ]
)

# Initialize the pipeline
pipeline = CoursePipeline()
# Method 1: Using config file
course = pipeline.process_from_config("configs/cours_1.yaml")


# Get processing statistics
stats = pipeline.get_processing_statistics(course)
print(f"Processed {stats['total_sections']} sections from {stats['total_slides']} slides")

2025-08-18 13:10:15,439 - pdfminer.pdfinterp - WARNING - Cannot set gray non-stroke color because /'P412' is an invalid float value
2025-08-18 13:10:15,440 - pdfminer.pdfinterp - WARNING - Cannot set gray non-stroke color because /'P414' is an invalid float value
2025-08-18 13:10:15,455 - pdfminer.pdfinterp - WARNING - Cannot set gray non-stroke color because /'P444' is an invalid float value
2025-08-18 13:10:15,470 - pdfminer.pdfinterp - WARNING - Cannot set gray non-stroke color because /'P451' is an invalid float value
2025-08-18 13:10:15,472 - pdfminer.pdfinterp - WARNING - Cannot set gray non-stroke color because /'P455' is an invalid float value
2025-08-18 13:10:15,488 - pdfminer.pdfinterp - WARNING - Cannot set gray non-stroke color because /'P465' is an invalid float value
2025-08-18 13:10:15,490 - pdfminer.pdfinterp - WARNING - Cannot set gray non-stroke color because /'P467' is an invalid float value
2025-08-18 13:10:15,512 - pdfminer.pdfinterp - WARNING - Cannot set gray non

💾 Course saved to: /Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/architecture_du_génome_humain_20250818_132011.json


2025-08-18 13:20:11,906 - src.pipeline - INFO - 💾 Course saved to DOCX
2025-08-18 13:20:11,906 - src.pipeline - INFO - 🎉 Course processing complete!


📄 Course exported to DOCX: /Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/docx/architecture_du_génome_humain.docx
Processed 7 sections from 110 slides


In [4]:
print(course.print_outline())

Course: Architecture du génome humain
Level: L1
Block: SANTE
Semester: S1
Subject: UE-1
Chapter: 1


Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
Description des principaux éléments constituants
  Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


In [6]:
import os

jsons = os.listdir("/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/")
file_path = "/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/" + jsons[-1]

jsons

['architecture_du_génome_humain_20250818_113431.json',
 'architecture_du_génome_humain_20250818_123215.json',
 'réplication_et_réparation_de_l’adn_20250818_120405.json',
 'architecture_du_génome_humain_20250818_114200.json',
 'architecture_du_génome_humain_20250818_132011.json']

In [7]:
from src.course import Course

course = Course.load_from_json(
file_path)
course

Course(name='Architecture du génome humain', course_title='Architecture du génome humain', level='L1', block='SANTE', semester='S1', subject='UE-1', chapter=1, content=Content(sections=[ContentSection(id='SEC_1', title='Introduction', content=["Ce module aborde les molécules du vivant en plaçant le génome au centre d'un cadre plus large qui comprend également le transcriptome, le protéome et le méthylome. Le programme couvre des notions fondamentales d'évolution et d'homologie, la structure et l'architecture du génome humain (nucléaire et mitochondrial), le projet de séquençage, la description des éléments constitutifs du génome, la comparaison avec d'autres génomes, ainsi que les types et les conséquences des variations génomiques.", "Le génome est présenté comme une référence stable de l'information héréditaire d'un organisme, servant de base de comparaison pour les autres couches fonctionnelles (expression, modifications épigénétiques, production protéique). L'étude intégrée de ces 

In [10]:
course.subject = "UE-2"
course.chapter = "7.1"

In [11]:
print(course.print_outline())

Course: Architecture du génome humain
Level: L1
Block: SANTE
Semester: S1
Subject: UE-2
Chapter: 7.1


Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
Description des principaux éléments constituants
  Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


In [7]:
course.save_to_json(output_path="volume/artifacts/json")

💾 Course saved to: volume/artifacts/json/architecture_du_génome_humain_20250818_114200.json


'volume/artifacts/json/architecture_du_génome_humain_20250818_114200.json'

In [12]:
atifacts_path = "volume/artifacts/docx"
template_path = "volume/templates/fs_template.docx"
course.to_docx(output_path=atifacts_path, template_path=template_path)

📄 Course exported to DOCX: volume/artifacts/docx/architecture_du_génome_humain.docx


'volume/artifacts/docx/architecture_du_génome_humain.docx'

## Step 1: Data Extraction - Slides

Extract slides from PDF using the exact logic from poc.ipynb


In [ ]:
# Method 2: Direct method call
metadata = CourseMetadata(
    name="Architecture du génome humain",
    course_title="Architecture du génome humain",
    level="L1",
    block="SANTE",
    semester="S1",
    subject="UE-1 Constitution et transformation de la matière"
)

course = pipeline.process_course_no_plan(
    slides=slides,  # Your slides list
    metadata=metadata,
    save_json=True,
    save_docx=True,
    test_mode=True,  # Process only first 10 slides
    output_path=""
)

# Get processing statistics
stats = pipeline.get_processing_statistics(course)
print(f"Processed {stats['total_sections']} sections from {stats['total_slides']} slides")